# Surameria Results

Cargamos los resultados de iterar todos los conjuntos de indicadores

In [2]:
import pandas as pd
import numpy as np
import time
import plotly.plotly as py
import plotly.graph_objs as go

original = pd.read_csv("suramerica_results_original.csv").drop(['Unnamed: 0'], 1)
random   = pd.read_csv("suramerica_results_random.csv").drop(['Unnamed: 0'], 1)
grid     = pd.read_csv("suramerica_results_grid.csv").drop(['Unnamed: 0'], 1)

datos = pd.read_csv("suramerica.csv").drop('Unnamed: 0', 1)
paises = datos['CountryCode'].drop_duplicates().values.tolist()
pais_nombre = 'Suramerica'

preprocessing = 'imput'
search = 'original'
year_init = 2012
year_range = 5
look_back = 3
years = range(year_init-year_range+1,year_init+1)[::-1]

agricultura     = open("Indicadores/iagricultura.txt").read().split(',')
ambiente        = open("Indicadores/iambiente.txt").read().split(',')
ayuda           = open("Indicadores/iayuda.txt").read().split(',')
ciencia         = open("Indicadores/iciencia.txt").read().split(',')
clima           = open("Indicadores/iclima.txt").read().split(',')
comercio        = open("Indicadores/icomercio.txt").read().split(',')
deuda           = open("Indicadores/ideuda.txt").read().split(',')
economia        = open("Indicadores/ieconomia.txt").read().split(',')
educacion       = open("Indicadores/ieducacion.txt").read().split(',')
energia         = open("Indicadores/ienergia.txt").read().split(',')
finanzas        = open("Indicadores/ifinanzas.txt").read().split(',')
genero          = open("Indicadores/igenero.txt").read().split(',')
infraestructura = open("Indicadores/iinfraestructura.txt").read().split(',')
pobreza         = open("Indicadores/ipobreza.txt").read().split(',')
privado         = open("Indicadores/iprivado.txt").read().split(',')
publico         = open("Indicadores/ipublico.txt").read().split(',')
salud           = open("Indicadores/isalud.txt").read().split(',')
social          = open("Indicadores/isocial.txt").read().split(',')
trabajo         = open("Indicadores/itrabajo.txt").read().split(',')
urbano          = open("Indicadores/iurbano.txt").read().split(',')


conjunto_nombre = ['Agricultura','Ambiente','Ayuda','Ciencia','Clima','Comercio','Deuda','Economia','Educacion',
                   'Energia','Finanzas','Genero','Infraestructura','Pobreza','Privado','Publico','Salud','Social',
                   'Trabajo','Urbano']
conjunto = [agricultura,ambiente,ayuda,ciencia,clima,comercio,deuda,economia,educacion,energia,finanzas,genero,
            infraestructura,pobreza,privado,publico,salud,social,trabajo,urbano]

In [3]:
def tabla_base(indicadores):
    tab = pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False])
    return tab

In [4]:
def buscar_mejores(df):
    return df[df.Resultados > 0.9].reset_index(drop=True)

In [5]:
def graficar_resultados(df,look_back):

    trace = go.Heatmap(z=df[df['Look Back'] == look_back]['%'],
                       x=df[df['Look Back'] == look_back]['Base'],
                       y=df[df['Look Back'] == look_back]['Target'],
                       zmax=1.0,
                       zmin=0.0,
                       colorscale = 'Viridis')
    data=[trace]
    
    layout = go.Layout(
        title=pais_nombre+' + '+search+': Total Heatmap with Look Back = '+str(look_back),
        xaxis=dict(title='Base',   titlefont=dict(size=18)),
        yaxis=dict(title='Target', titlefont=dict(size=18))
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig,filename=pais_nombre+' + '+search+': Total Heatmap with Look Back = '+str(look_back))

In [6]:
def buscar_resultados(df,indicador_1,indicador_2,look_back):
    #Buscamos Resultados en base al conjunto de indicadores Base, Target y Look Back
    temp = df[(df['Base']==indicador_1)&(df['Target']==indicador_2)&(df['Look Back'] == look_back)]['Results'].values[0]
    df_temp = temp.replace("'","").replace("[","").replace("]","").replace(" ","").split('\n')
    results = []
    for i in df_temp: 
        results.append(float(i))
    
    # Códigos de los indicadores con registros 
    df_nombres = pd.DataFrame(conjunto_nombre, columns=['Resultados'])
    indicador_1 = conjunto[df_nombres[df_nombres['Resultados']==indicador_1].index.values[0]]
    indicador_2 = conjunto[df_nombres[df_nombres['Resultados']==indicador_2].index.values[0]]     
    df = pd.DataFrame(indicador_2)
    
    # Filtro indicadores repetidos
    if(indicador_1!=indicador_2):
        indicador_2 = np.array(df.loc[~df.ix[:,0].isin(indicador_1)]).flatten()
    df_zeros = tabla_base(indicador_2).dropna(how='all',axis=1)
    icodes = df_zeros.columns.values

    # Buscamos Nombres de los Códigos de los Indicadores
    nombres = pd.DataFrame(datos).loc[:,['IndicatorName','IndicatorCode']].set_index('IndicatorCode').loc[icodes].drop_duplicates()

    # Concatenamos Resultados y Nombres, Reindexando en base a los nombres
    df_results = pd.DataFrame(np.column_stack([nombres,results]),columns=['Indicadores','Resultados'])
    
    return df_results

In [7]:
def graficar_resultados_individuales(df,indicador_base,indicador_target,look_back):

    df_temp = buscar_resultados(df,indicador_base,indicador_target,look_back)

    data = [go.Bar(
                x= df_temp['Indicadores'],
                y= df_temp['Resultados'],

        marker=dict(     
            color = df_temp['Resultados'].values,
            colorscale ='Viridis',
            showscale = True
            ),
        )]

    layout = go.Layout(
        title=pais_nombre+' + '+search+': Result to predict '+indicador_target+' from '+indicador_base+' with '+'Look Back = '+str(look_back),
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig, filename=pais_nombre+' + '+search+': Result to predict '+indicador_target+' from '+indicador_base+' with '+'Look Back = '+str(look_back))

In [8]:
def conjuntos_predecibles(df,look_back):
    df_temp = pd.DataFrame.pivot_table( df[df['Look Back'] == 1].drop(['Time','Reg','Results'],1) , index=["Target","Base"], columns=["Look Back"])
    return pd.DataFrame(df_temp[df_temp['%'] > 0.3].count(level=0).sort_values([('%',1)], ascending=False).index.values, columns=["Conjunto"])

In [9]:
def conjuntos_predictores(df,look_back):
    df_temp = pd.DataFrame.pivot_table( df[df['Look Back'] == 1].drop(['Time','Reg','Results'],1) , index=["Base","Target"], columns=["Look Back"])
    return pd.DataFrame(df_temp[df_temp['%'] > 0.3].count(level=0).sort_values([('%',1)], ascending=False).index.values, columns=["Conjunto"])

## Buscamos Mejores Resultados Individuales

In [41]:
buscar_mejores(buscar_resultados(df,'Agricultura','Infraestructura',1))

Indicadores Resultados
0             Air transport, freight (million ton-km)   0.910649
1   Container port traffic (TEU: 20 foot equivalen...   0.957681
2         Electric power consumption (kWh per capita)    0.91501
3   Electricity production from hydroelectric sour...   0.920372
4   Electricity production from natural gas source...   0.963087
5   Electricity production from oil sources (% of ...   0.936279
6                       Fixed broadband subscriptions   0.918652
7                       Fixed telephone subscriptions    0.99226
8      Fixed telephone subscriptions (per 100 people)     0.9844
9              ICT service exports (BoP, current US$)   0.936051
10  Investment in transport with private participa...   0.951249
11                      Mobile cellular subscriptions   0.914159
12                        Rail lines (total route-km)   0.929643
13       Railways, goods transported (million ton-km)   0.999056
14  Railways, passengers carried (million passenge...   0.906451
15     Secure Internet servers (per 1 million people)   0.904757

## Mejores combinaciones

In [42]:
df.sort_values('%', ascending=False).reset_index().drop(['index','Time','Results'],1)[:10]

Base   Target  Look Back  Reg         %
0          Privado  Ciencia          1   11  0.900546
1           Urbano  Ciencia          2   13  0.892362
2         Comercio  Ciencia          2   11  0.879526
3         Comercio  Ciencia          3   11  0.877208
4  Infraestructura  Ciencia          2   13  0.874702
5          Energia  Ciencia          1   13  0.862954
6          Ciencia  Ciencia          3   13  0.855968
7          Privado  Ciencia          2   11  0.855853
8           Urbano  Ciencia          1   13  0.842303
9         Comercio  Ciencia          1   11  0.840930

## Peores combinaciones

In [43]:
df.sort_values('%', ascending=False).reset_index().drop(['index','Time','Results'],1).tail(10)

Base    Target  Look Back  Reg         %
1190  Pobreza   Energia          3   53  0.225532
1191  Pobreza  Ambiente          1  111  0.223511
1192  Pobreza     Salud          3  152  0.215839
1193  Pobreza     Salud          2  152  0.210652
1194  Pobreza    Urbano          2   21  0.210160
1195  Pobreza     Ayuda          3   69  0.208370
1196  Pobreza     Salud          1  152  0.201384
1197  Pobreza     Ayuda          2   69  0.195450
1198  Pobreza    Urbano          1   21  0.192287
1199  Pobreza    Urbano          3   21  0.171713

## Graficamos Resultados Individuales

Resultado de predecir el conjunto de indicadores de Infraestructura desde Agricultura

In [44]:
graficar_resultados_individuales(df,'Agricultura','Infraestructura',1)

In [45]:
graficar_resultados_individuales(df,'Agricultura','Economia',1)

## Graficamos Resultados Globales

Este Heatmap nos muestra el rendimiento promedio del coeficiente de determinación para cada conjunto

In [46]:
graficar_resultados(df,1)

In [47]:
graficar_resultados(df,2)

In [48]:
graficar_resultados(df,3)

# Conclusiones

- Con el objetivo de encontrar los conjuntos con mayor grado de predictibilidad, contamos la cantidad de conjuntos en los cuales el 30% o más de sus indicadores obtuvieron un puntaje mayor al 90% para dos casos:

 **Predictor:** Conjunto que predice a otro conjunto
 
 **Predecible:** Conjunto que es predecido por otro conjunto

In [49]:
pd.DataFrame(np.column_stack([conjuntos_predictores(df,1)[:5],conjuntos_predictores(df,1).tail(5),conjuntos_predecibles(df,1)[:5],conjuntos_predecibles(df,1).tail(5)]), columns=['Best Predictor','Worse Predictor','Easier Predictable','Hard Predictable'])

Best Predictor Worse Predictor Easier Predictable Hard Predictable
0    Agricultura           Clima        Agricultura            Ayuda
1       Ambiente         Ciencia           Economia            Salud
2        Trabajo           Ayuda            Publico           Social
3         Social          Urbano            Pobreza          Trabajo
4          Salud         Pobreza           Finanzas           Urbano